In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('nba.db')
conn.execute("""ATTACH 'covid19.db' AS covid19""")
conn.execute("""ATTACH 'twElection2020.db' AS twElection2020""")
conn.execute("""ATTACH 'imdb.db' AS imdb""")

## 46. 從 covid19 資料庫查詢兩艘郵輪（Grand Princess 與 Diamond Princess）的資訊

In [3]:
find_cruise_ships_from_covid19 =\
"""
-- SQL 查詢語法起點

SELECT lookup_table.iso2,
       lookup_table.Country_Region,
       lookup_table.Province_State,
       daily_report.Confirmed
  FROM lookup_table
 JOIN daily_report
   ON daily_report.Combined_Key = lookup_table.Combined_Key
 WHERE lookup_table.Province_State IN ('Grand Princess', 'Diamond Princess') 

-- SQL 查詢語法終點
"""

cruise_ships_from_covid19 = pd.read_sql(find_cruise_ships_from_covid19, conn)
cruise_ships_from_covid19

,iso2,Country_Region,Province_State,Confirmed
0,CA,Canada,Diamond Princess,0
1,CA,Canada,Grand Princess,13
2,US,US,Diamond Princess,49
3,US,US,Grand Princess,103


## 47. 從 covid19 資料庫查詢截至 2021-03-31 所有國家確診與死亡人數的資訊

In [4]:
summarize_confirmed_deaths_by_country_from_covid19 =\
"""
-- SQL 查詢語法起點

SELECT lookup_table.Country_Region,
       SUM(daily_report.Confirmed) AS Confirmed,
       SUM(daily_report.Deaths) AS Deaths
  FROM daily_report
 JOIN lookup_table
   ON lookup_table.Combined_Key = daily_report.Combined_Key
 GROUP BY lookup_table.Country_Region;

-- SQL 查詢語法終點
"""

confirmed_deaths_by_country_from_covid19 = pd.read_sql(summarize_confirmed_deaths_by_country_from_covid19, conn)
confirmed_deaths_by_country_from_covid19

,Country_Region,Confirmed,Deaths
0,Afghanistan,56454,2484
1,Albania,125157,2235
2,Algeria,117192,3093
3,Andorra,12010,115
4,Angola,22311,537
...,...,...,...
187,Vietnam,2603,35
188,West Bank and Gaza,242353,2627
189,Yemen,4357,888
190,Zambia,88418,1208


## 48. 從 imdb 資料庫查詢「魔戒三部曲」與「蝙蝠俠三部曲」的電影資訊與演員名單，三部曲電影系列中演員重複出演的情況是正常的，這時顯示獨一值即可

* 先把所有會用到的資料表做水平合併，再決定要取哪一個資訊

In [5]:
find_two_trilogy_casting_list =\
"""
-- SQL 查詢語法起點

SELECT DISTINCT CASE WHEN movies.title LIKE 'The Lord of the Rings%' THEN 'The Lord of the Rings Trilogy'
                ELSE 'Batman Trilogy' END AS trilogy,
       actors.name AS name
  FROM casting
 JOIN movies
   ON casting.movie_id = movies.id
 JOIN actors
   ON casting.actor_id = actors.id
 WHERE movies.title LIKE '%Batman%' OR
       movies.title LIKE '%The Lord of the Rings%' OR
       movies.title LIKE 'The Dark Knight%'
 ORDER BY trilogy,
          name;

-- SQL 查詢語法終點
"""
two_trilogy_casting_list = pd.read_sql(find_two_trilogy_casting_list, conn)
two_trilogy_casting_list

,trilogy,name
0,Batman Trilogy,Aaron Eckhart
1,Batman Trilogy,Aidan Gillen
2,Batman Trilogy,Alon Aboutboul
3,Batman Trilogy,Anne Hathaway
4,Batman Trilogy,Anthony Michael Hall
...,...,...
61,The Lord of the Rings Trilogy,Sadwyn Brophy
62,The Lord of the Rings Trilogy,Sala Baker
63,The Lord of the Rings Trilogy,Sam Comery
64,The Lord of the Rings Trilogy,Sean Astin


## 49. 從 nba 資料庫查詢截至 2021-03-31 的得分王（生涯場均得分 ppg 最高）、助攻王（生涯場均助攻 apg 最高）、籃板王（生涯場均籃板 rpg 最高）、抄截王（生涯場均抄截 spg 最高）以及阻攻王（生涯場均阻攻 bpg 最高）

* 很明顯的垂直合併題

In [6]:
find_max_stats_per_game_from_nba =\
"""
-- SQL 查詢語法起點

SELECT players.firstName,
       players.lastName,
       'rpg' AS category,
       career_summaries.rpg AS value
  FROM career_summaries
 JOIN players
   ON career_summaries.personId = players.personId
 WHERE career_summaries.rpg = (SELECT MAX(career_summaries.rpg)
                                 FROM career_summaries)
UNION
SELECT players.firstName,
       players.lastName,
       'bpg' AS category,
       career_summaries.bpg AS value
  FROM career_summaries
 JOIN players
   ON career_summaries.personId = players.personId
  WHERE career_summaries.bpg = (SELECT MAX(career_summaries.bpg)
                                  FROM career_summaries)
UNION
SELECT players.firstName,
       players.lastName,
       'apg' AS category,
       career_summaries.apg AS value
  FROM career_summaries
 JOIN players
   ON career_summaries.personId = players.personId
  WHERE career_summaries.apg = (SELECT MAX(career_summaries.apg)
                                  FROM career_summaries)
UNION
SELECT players.firstName,
       players.lastName,
       'spg' AS category,
       career_summaries.spg AS value
  FROM career_summaries
 JOIN players
   ON career_summaries.personId = players.personId
  WHERE career_summaries.spg = (SELECT MAX(career_summaries.spg)
                                  FROM career_summaries)
UNION
SELECT players.firstName,
       players.lastName,
       'ppg' AS category,
       career_summaries.ppg AS value
  FROM career_summaries
 JOIN players
   ON career_summaries.personId = players.personId
  WHERE career_summaries.ppg = (SELECT MAX(career_summaries.ppg)
                                  FROM career_summaries)
 ORDER BY firstName;


-- SQL 查詢語法終點
"""

max_stats_per_game_from_nba = pd.read_sql(find_max_stats_per_game_from_nba, conn)
max_stats_per_game_from_nba

,firstName,lastName,category,value
0,Andre,Drummond,rpg,13.8
1,Anthony,Davis,bpg,2.4
2,Chris,Paul,apg,9.4
3,Chris,Paul,spg,2.2
4,Hassan,Whiteside,bpg,2.4
5,Kevin,Durant,ppg,27.1
